<a href="https://colab.research.google.com/github/byui-cse/cse480-notebooks/blob/master/07_1_Assorted_Topics_from_Chapters_11__and_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assorted Topics from Chapters 11 and 12
## Ponder and Prove
### Due: Saturday, 20 February 2021, 11:59 pm

## The goal is for you to make some good connections between these formal objects:
 
* Parse trees
* Derivation Sequences
* Instantaneous Description Sequences
 
See the example in Figure 11.1, with the derivation sequence shown on the right of this figure, and the ID sequence also shown below.

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:

  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty',
                   '../Jove',      '../Jove/jove']

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.SystemImports import *
from jove.DotBashers    import *
from jove.Def_md2mc     import *
from jove.Def_PDA       import * 
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Dyck CFG and Direct-Converted PDA

 Note that the CFG is inside the PDA code as a comment.

In [ ]:
dyck = md2mc('''
PDA
!!-- CFG follows
!!-- S -> SS | (S) | ''

IF : '',#; S# -> W

W  : (,(; '' -> W
W  : ),); '' -> W

W  : b,b; bb -> W

W  : '',S; SS  -> W
W  : '',S; (S) -> W
W  : '',S; ''  -> W

W  : '',#; # -> F

''')
dotObj_pda(dyck, FuseEdges=True)

### TODO: Convert and compare Jove with the Automaton Simulator

This is the one demonstrated in class last week:

https://automatonsimulator.com/

Include a link encoding your saved machine.

In [ ]:
explore_pda('((())())()', dyck)

## Derivation Sequences (DS) are Formed from Instantaneous Description (ID) Sequences

Note that Derivation sequences (DS) are "flattened parse trees"!

Here's how to reverse-engineer DS from the ID sequence printed:

```
     ID SEQUENCE                 -- FORMING EACH STEP of DERIVATION SEQUENCE
=================================================================================     
->  ('IF', '((())())()', '#')    -- 
->  ('W', '((())())()', 'S#')    -- S
->  ('W', '((())())()', 'SS#')   -- S     => SS      (on top of stack)
->  ('W', '((())())()', '(S)S#') -- SS    => (S)S    (push on TOS; record in deriv seq
->  ('W', '(())())()', 'S)S#')   -- (S)S  => popped  (so no new derivation; leaf-match in PT)
->  ('W', '(())())()', 'SS)S#')  -- (S)S  => (SS)S   (new push; so record in deriv. sequence)
->  ('W', '(())())()', '(S)S)S#')-- (SS)S => ((S)S)S (again notice new push!)
```

Thus, we note that S => SS => (S)S => (SS)S => ((S)S)S => ... ((())())() 




 

### TODO Draw the Parse Tree from this DS.

Use a neat drawing tool, and put the source and the PNG here.

## Disambiguated Expression Parsing

Derivation sequence for ```2*3+2``` noted below

In [ ]:
# Parsing an arithmetic expression
pdaE = md2mc('''PDA

!!-- The grammar E -> E+E | E*E | (E) | ~E | 2 | 3 is ambiguous!
!!-- Two parse trees!
!!-- This is your Asg Part 2b

!!-- STRATIFIED GRAMMAR for expressions avoids ambiguity - below!
!!E -> E+T | T
!!T -> T*F | F
!!F -> 2 | 3 | ~F | (E)
!!-- Direct-converted PDA from above CFG
I : '', #  ; E#  -> M
M : '', E  ; E+T -> M
M : '', E  ; T   -> M
M : '', T  ; T*F -> M
M : '', T  ; F   -> M
M : '', F  ; 2   -> M
M : '', F  ; 3   -> M
M : '', F  ; ~F  -> M
M : '', F  ; (E) -> M
M : ~,  ~  ; ''  -> M
M : 2,  2  ; ''  -> M
M : 3,  3  ; ''  -> M
M : (,  (  ; ''  -> M
M : ),  )  ; ''  -> M
M : +,  +  ; ''  -> M
M : *,  *  ; ''  -> M
M : '', #  ; #   -> F
'''
)

### TODO: Convert and compare Jove with the Automaton Simulator

As before. Include a link encoding your saved machine.

In [ ]:
explore_pda('2*3+2', pdaE)

### ID Sequence to Derivation Sequence to Implied Parse-Tree

```
     ID SEQUENCE             -- FORMING EACH STEP of DERIVATION SEQUENCE
=================================================================================  
->  ('I', '2*3+2', '#')      -- 
->  ('M', '2*3+2', 'E#')     -- E        (pushed)
->  ('M', '2*3+2', 'E+T#')   -- => E+T   (removed E; pushed E+T)
->  ('M', '2*3+2', 'T+T#')   -- => T+T   (pushed T)
->  ('M', '2*3+2', 'T*F+T#') -- => T*F+T (removed T, pushed T*F)
->  ('M', '2*3+2', 'F*F+T#') -- => F*F+T (removed T, pushed F)
->  ('M', '2*3+2', '2*F+T#') -- => 2*F+T (removed F, pushed 2)
->  ('M', '*3+2', '*F+T#')   -- => leaf-match of 2, no new push, deriv seq same as before
->  ('M', '3+2', 'F+T#')     -- => leaf-match of *, no new push. NOTICE "*" DONE BEFORE +
->  ('M', '3+2', '3+T#')     -- => ...
->  ('M', '+2', '+T#')
->  ('M', '2', 'T#')
->  ('M', '2', 'F#')
->  ('M', '2', '2#')
->  ('M', '', '#')
->  ('F', '', '#') . 
```


### TODO: Finish --- Derivation sequence is


```E => E+T => T+T => T*F+T => F*F+T => 2*F+T => ... 2*3+2``` (You flesh out.)

### TODO Draw the Parse Tree from this DS.

Use a neat drawing tool, and put the source and the PNG here.

Note how * is done before +.

## TODO Study Context-Free Grammars Further



1. Learn how to "run" CFGs by directly converting them to PDA, and running those PDA.
2. Watch YouTube videos as presented "in context" below.

#### CFG Basics and How to Directly Parse Them Using PDA

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('bymg3MDe6Y4')

Direct path: https://youtu.be/bymg3MDe6Y4 (31 min mark).

From about the 30-minute mark, the video discusses the grammar:

S -> aSa | bSb | epsilon,

for $\{w w^R \;:\; w \in \{a, b\}^* \}$.

### The Next Video

(Direct path https://youtu.be/N5wQovKIZGM) discusses:
1. Consistency/Completeness
2. Linearity
3. Ambiguity

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('N5wQovKIZGM')

### TODO Run the "Direct Simulation of CFG from PDA"

for the above grammar. Notice how each grammar rule is encoded DIRECTLY as a PDA transition.

These PDA always have the *SAME* I to W and W to F moves ...

Only the W to W moves vary; and here, they have transitions that go by the CFG.

There are always $a \in \Sigma$ transitions that drain every $a \in \Sigma$, matching it with an incoming $a$.

In [ ]:
wwR = md2mc('''
PDA
!!-- The CFG to be simulated
!!-- S -> aSa | bSb | ''

!!-- This move is the same for any CFG
I : '',#; S# -> W

!!-- These moves imitate the CFG productions directly
W : '',S; aSa -> W
W : '',S; bSb -> W
W : '',S; ''  -> W

!!-- Transitions for every member of Sigma
W : a,a; '' -> W
W : b,b; '' -> W

!!-- This transition is also there for every CFG
W : '',#; ''   -> F
''')
dotObj_pda(wwR, FuseEdges=True)

In [ ]:
explore_pda('aabaabaa', wwR)

### Animate with Jove and Alternative

In [ ]:
from jove.AnimatePDA import *
AnimatePDA(wwR, FuseEdges=True)
display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))


Do this and all the rest with the alternative Automaton Simulator also. https://automatonsimulator.com

### Animation Panel for all Machines (choose PDA)

### You may do the above animation here.
#### Access the PDA panel and key-in the definition of wwR in there.

In [ ]:
from jove.JoveEditor import *
JoveEditor(examples=True)
display(HTML('<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"/>'))


The explore_pda call below fails because the string is not of the form $ww^R$. Here you must use ```explore_pda```, not animation, to further investigate.

In [ ]:
explore_pda('aabaaaba', wwR)

### An RE Grammar and its PDA Encoding

Observe the number of possible parses.

This is a highly ambiguous grammar because there are many parses for the same input.

In [ ]:
repda = md2mc('''PDA
!!R -> R R | R + R | R* | ( R ) | 0 | 1 | e
I : '', #  ; R#  -> M
M : '', R  ; RR  -> M
M : '', R  ; R+R -> M
M : '', R  ; R*  -> M
M : '', R  ; (R) -> M
M : '', R  ; 0   -> M
M : '', R  ; 1   -> M
M : '', R  ; e   -> M
M : 0,  0  ; ''  -> M
M : 1,  1  ; ''  -> M
M : (,  (  ; ''  -> M
M : ),  )  ; ''  -> M
M : +,  +  ; ''  -> M
M : e,  e  ; ''  -> M
M : '', #  ; #   -> F
'''
)

In [ ]:
repda

In [ ]:
DO_repda = dotObj_pda(repda, FuseEdges=True)

In [ ]:
DO_repda

In [ ]:
explore_pda("0", repda, STKMAX=4)

In [ ]:
explore_pda("00", repda)

In [ ]:
explore_pda("(0)", repda)

In [ ]:
explore_pda("(00)", repda)

In [ ]:
explore_pda("(0)(0)", repda)

In [ ]:
explore_pda("0+0", repda, STKMAX=3)

In [ ]:
explore_pda("(0)+(0)", repda)

In [ ]:
explore_pda("00+0", repda)

In [ ]:
explore_pda("000", repda, STKMAX=3)

In [ ]:
explore_pda("00+00", repda, STKMAX=4)

In [ ]:
explore_pda("00+00", repda, STKMAX=5)

### PDA for Well-Balanced Parentheses

In [ ]:
brpda1 = md2mc('''PDA

!!-- The CFG simulated is
!!-- S -> (S)|SS|''

!!-- Initial move to state M (was W above)
 I : '', '' ; S     -> M
 
 M : '', S  ; (S) -> M
 M : '', S  ; SS   -> M
 M : '', S  ; ''    -> M
 
 M : (,  (  ; ''    -> M
 M : ),  )  ; ''    -> M
 
 M : '',  ''  ; ''  -> M
 
 M : '', #  ; ''    -> F''')

dotObj_pda(brpda1, FuseEdges=True)

In [ ]:
explore_pda("()(())", brpda1, STKMAX=5)

### Ambiguous Parsing of Arithmetic Expressions

In [ ]:
# Parsing an arithmetic expression
pdaEamb = md2mc('''PDA
!!E -> E * E | E + E | ~E | ( E ) | 2 | 3
I : '', #  ; E#  -> M
M : '', E  ; ~E  -> M
M : '', E  ; E+E -> M
M : '', E  ; E*E -> M
M : '', E  ; (E) -> M
M : '', E  ; 2   -> M
M : '', E  ; 3   -> M
M : ~,  ~  ; ''  -> M
M : 2,  2  ; ''  -> M
M : 3,  3  ; ''  -> M
M : (,  (  ; ''  -> M
M : ),  )  ; ''  -> M
M : +,  +  ; ''  -> M
M : *,  *  ; ''  -> M
M : '', #  ; #   -> F
'''
)

In [ ]:
DOpdaEamb = dotObj_pda(pdaEamb, FuseEdges=True)
DOpdaEamb

In [ ]:
explore_pda("3+2*3", pdaEamb, STKMAX=5)

In [ ]:
explore_pda("3+2*3+2*3", pdaEamb, STKMAX=7)

### Disambiguated Arithmetic Expression Grammar

In [ ]:
# Parsing an arithmetic expression
pdaE = md2mc('''PDA
!!E -> E+T | T
!!T -> T*F | F
!!F -> 2 | 3 | ~F | (E)
I : '', #  ; E#  -> M
M : '', E  ; E+T -> M
M : '', E  ; T   -> M
M : '', T  ; T*F -> M
M : '', T  ; F   -> M
M : '', F  ; 2   -> M
M : '', F  ; 3   -> M
M : '', F  ; ~F  -> M
M : '', F  ; (E) -> M
M : ~,  ~  ; ''  -> M
M : 2,  2  ; ''  -> M
M : 3,  3  ; ''  -> M
M : (,  (  ; ''  -> M
M : ),  )  ; ''  -> M
M : +,  +  ; ''  -> M
M : *,  *  ; ''  -> M
M : '', #  ; #   -> F
'''
)

In [ ]:
DOpdaE = dotObj_pda(pdaE, FuseEdges=True)
DOpdaE

In [ ]:
explore_pda("3+2*3", pdaE, STKMAX=7)

In [ ]:
explore_pda("3+2*3+2*3", pdaE, STKMAX=7)

In [ ]:
explore_pda("3*2*~3+~~3*~3", pdaE, STKMAX=10)

In [ ]:
explore_pda("3*2*~3+~~3*~3", pdaEamb, STKMAX=8)